In [ ]:
import os
import glob
import numpy as np
from tensorflow.keras.utils import  plot_model
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from PIL import Image, ImageDraw
import pandas as pd
import seaborn as sns
from keras.callbacks import ModelCheckpoint
import keras.callbacks
import os
import pydot
os.chdir()

In [ ]:
# Image Parameters
N_CLASSES = 2 # total number of classes
IMG_HEIGHT = 320 # the image height to be resized to
IMG_WIDTH = 320 # the image width to be resized to
CHANNELS = 3 # The 3 color channels, change to 1 if grayscale
initial_train_data = 
initial_validation = 
tf_train_data = 
tf_validation = 

In [ ]:

#rewriting darknet dataset to ft2 folder format
import cv2
with open(os.path.join(tf_train_data,'train_dataset.txt'),'w') as new_txt:
    for image in glob.glob(os.path.join(initial_train_data,'*.jpg')):
        title, ext = os.path.splitext(os.path.basename(image))
        txt_file_name = title + '.txt'
        cv_image = cv2.imread(image)
        cv2.imwrite(tf_train_data + '/' + title + '.jpg',cv_image)
        with open(os.path.join(initial_train_data,txt_file_name),'r') as old_txt:
            labelList = old_txt.readlines()
            for label in labelList:
                label = label.strip().split()
                class_ = int(label[0])
                x_c = float(label[1])
                y_c = float(label[2])
                h   = float(label[4])
                w   = float(label[3])
        
            new_txt.write(tf_train_data + '/' + title + '.jpg' + '\t' + str(int(label[0])) + '\t' + str(x_c) 
            + '\t' + str(y_c) + '\t' + str(w) + '\t' + str(h) + '\n')


In [ ]:
from keras.utils import to_categorical
def parse_dataset(dataset_path, ext='jpg'):
    """
    Used to extract information about our dataset. It does iterate over all images and return a DataFrame with
    the data (age, gender and sex) of all files.
    """
    def parse_info_from_file(path):
        """
        Parse information from a single file
        """
        try:
            title, ext = os.path.splitext(os.path.basename(path))
            txt_file_name = title + '.txt'
            with open(os.path.join(os.path.dirname(os.path.abspath(path)),txt_file_name),'r') as old_txt:
                labelList = old_txt.readlines()
                #print(str(labelList))
                for label in labelList:
                    label = label.strip().split()
                    class_= int(label[1])
                    x_c   = float(label[2])
                    y_c   = float(label[3])
                    w     = float(label[4])
                    h     = float(label[5])
            return class_, x_c, y_c, w, h
        except Exception as ex:
            return class_, None, None, None, None
        
    files = glob.glob(os.path.join(dataset_path, "*.%s" % ext))
    
    records = []
    for file in files:
        info = parse_info_from_file(file)
        records.append(info)
        
    df = pd.DataFrame(records)
    df['file'] = files
    df.columns = ['class', 'x_c', 'y_c', 'w', 'h', 'file']
    df = df.dropna()
    
    return df

df = parse_dataset(tf_train_data)
df.head()

In [ ]:
val_df = parse_dataset(tf_validation)
val_df.head()

In [ ]:
class DataGenerator():
    """
    Data generator for the UTKFace dataset. This class should be used when training our Keras multi-output model.
    """
    def __init__(self, df):
        self.df = df
        
    def generate_split_indexes(self):
        p = np.random.permutation(len(self.df))
        train_idx = p[:]
        
        return train_idx
    
    def generate_test_split_indexes(self,length):
        p = np.random.randint(0,len(self.df),length)
        train_idx = p[:]
        
        return train_idx
    
    def preprocess_image(self, img_path):
        """
        Used to perform some minor preprocessing on the image before inputting into the network.
        """
        
        im = Image.open(img_path)
        im = im.resize((IMG_WIDTH, IMG_HEIGHT))
        im = np.array(im) / 255.0
        return im
        
    def generate_images(self, image_idx, is_training, batch_size):
        """
        Used to generate a batch with images when training/testing/validating our Keras model.
        """
        
        # arrays to store our batched data
        images, classes, x_c, y_c, w, h = [], [], [], [], [], []
        while True:
            for idx in image_idx:
                imag = self.df.iloc[idx]
                
                classes_ = imag['class']
                x_c_ = imag['x_c']
                y_c_ = imag['y_c']
                w_ = imag['w']
                h_ = imag['h']
                file = imag['file']
                
                im = self.preprocess_image(file)
                
                classes.append(classes_)
                x_c.append(x_c_)
                y_c.append(y_c_)
                w.append(w_)
                h.append(h_)
                images.append(im)
                #print(np.array(y_c))
                # yielding condition
                if len(images) >= batch_size:
                    yield np.array(images), [np.array((classes)),np.array(x_c),np.array(y_c), np.array(w), np.array(h)]
                    images, classes, x_c, y_c, w, h = [], [], [], [], [], []
                    
            if not is_training:
                break
                
data_generator = DataGenerator(df)
train_idx = data_generator.generate_split_indexes() 
v_data_generator = DataGenerator(val_df)
valid_idx = v_data_generator.generate_split_indexes() 

In [ ]:
inputs = keras.Input(shape=(IMG_WIDTH, IMG_HEIGHT, CHANNELS), name="img_input")
#classifier
x = layers.Conv2D(16, (3, 3),activation='relu')(inputs)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Dropout(0.5)(x)
x = layers.Conv2D(16, (3, 3),activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Dropout(0.5)(x)
x = layers.Conv2D(32, (3, 3),activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Dropout(0.5)(x)
x = layers.Conv2D(32, (3, 3),activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Dropout(0.5)(x)
x = layers.Conv2D(32, (3, 3),activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Dropout(0.5)(x)
def_set = layers.Conv2D(64, (3, 3),activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Dropout(0.25)(def_set)
x = layers.Conv2D(64, (3, 3),activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Dropout(0.25)(x)
x = layers.Conv2D(64, (3, 3),activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Dropout(0.5)(x)
x = layers.Conv2D(64, (3, 3),activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Dropout(0.5)(x)
x = layers.Conv2D(64, (3, 3),activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Dropout(0.5)(x)
x = layers.Conv2D(64, (3, 3),activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Dropout(0.5)(x)
x = layers.Conv2D(64, (3, 3),activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Dropout(0.5)(x)
x = layers.Flatten()(x)
x = layers.Dense(128)(x)
x = layers.Activation("relu")(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
x = layers.Activation("sigmoid")(x)
x = layers.Dense(10)(x)
x = layers.Activation("sigmoid")(x)
x = layers.Dense(2)(x)
class_output = layers.Softmax(axis=-1,name="class_output")(x)
#x pos loc
x = layers.Conv2D(32, (3, 3),activation='relu')(def_set)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Dropout(0.5)(x)
x = layers.Conv2D(32, (3, 3),activation='relu')(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Dropout(0.5)(x)
x = layers.Conv2D(32, (3, 3), padding="same")(x)
x = layers.Activation("relu")(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Conv2D(32, (3, 3), padding="same")(x)
x = layers.Activation("relu")(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Conv2D(32, (3, 3), padding="same")(x)
x = layers.Activation("relu")(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.Conv2D(32, (3, 3), padding="same")(x)
x = layers.Activation("relu")(x)
x = layers.BatchNormalization(axis=-1)(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Flatten()(x)
Box_output = layers.Dense(1,name="x_loc_output")(x)


model2 = keras.Model(inputs, [class_output,Box_output], name="cats_dog")


In [ ]:
plot_model(model2,to_file='model.pdf',show_shapes=True)

In [ ]:
init_lr = 1e-3
epochs = 15
opt = Adam(lr=init_lr, decay=init_lr / epochs)
model2.compile(
    optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07),
     loss={
                  'Box_output': 'mse',
                  'class_output': 'sparse_categorical_crossentropy'},
     loss_weights={
                  'Box_output': 1,
                  'class_output': 0.25},
     metrics={
        "Box_output": [
            #keras.metrics.MeanAbsolutePercentageError(),
            keras.metrics.MeanAbsoluteError()],
        "class_output": [keras.metrics.CategoricalAccuracy()],
    },
)

In [ ]:
#%tensorboard --logdir log

In [ ]:
def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

LRS = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
batch_size = 24
valid_batch_size = 24
train_gen =   data_generator.generate_images(train_idx, is_training=True, batch_size=batch_size)
valid_gen = v_data_generator.generate_images(valid_idx, is_training=True, batch_size=valid_batch_size)
callbacks = [
    ModelCheckpoint(r"C:\Users\densh\Desktop\catdogs\keras_try\Checkpoint", monitor='val_loss'), #LRS,
    keras.callbacks.TensorBoard(
    log_dir=r"C:\Users\densh\Desktop\catdogs\keras_try\log",write_images=True,
    histogram_freq=0,  # How often to log histogram visualizations
    embeddings_freq=0,  # How often to log embedding visualizations
    update_freq="epoch")  # How often to write logs (default: once per epoch)
]

In [ ]:
model2.fit(train_gen,steps_per_epoch=len(train_idx)//(2*batch_size), callbacks=callbacks, epochs=15, validation_data=valid_gen, validation_steps=len(valid_idx)//valid_batch_size)

In [ ]:
model2.save(
    r"C:\Users\densh\Desktop\catdogs\keras_try\models",
    overwrite=True, include_optimizer=True, save_format=None,
    signatures=None, options=None, save_traces=True
